In [ ]:
from matplotlib.animation import FFMpegWriter
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# Enable inline plotting for Jupyter Notebook in VS Code
%matplotlib inline

# Constants
NUM_PARTICLES = 100
TIME_STEP = 0.1
PARTICLE_RADIUS = 0.5
CIRCLE_CENTER = np.array([0, 0])
CIRCLE_RADIUS = 5
FRAME_COUNT = 200

# Initialize particles
particles = {
    'position': np.random.rand(NUM_PARTICLES, 2) * 10 - 5,
    'velocity': np.random.randn(NUM_PARTICLES, 2)
}

def update_particles(particles):
    # Update particle positions
    particles['position'] += particles['velocity'] * TIME_STEP

    # Check for collision with circle
    for i in range(NUM_PARTICLES):
        dist_to_center = np.linalg.norm(particles['position'][i] - CIRCLE_CENTER)
        if dist_to_center < CIRCLE_RADIUS + PARTICLE_RADIUS:
            # Reflect velocity
            direction = (particles['position'][i] - CIRCLE_CENTER) / dist_to_center
            particles['velocity'][i] -= 2 * np.dot(particles['velocity'][i], direction) * direction

def animate(frame):
    update_particles(particles)
    plt.cla()
    circle = plt.Circle(CIRCLE_CENTER, CIRCLE_RADIUS, color='r', fill=False)
    plt.gca().add_patch(circle)
    plt.scatter(particles['position'][:, 0], particles['position'][:, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.gca().set_aspect('equal', adjustable='box')

# Create animation
fig = plt.figure()
ani = FuncAnimation(fig, animate, frames=FRAME_COUNT, interval=100)

# Display the animation in the Jupyter Notebook
HTML(ani.to_jshtml())


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Constants
NUM_PARTICLES = 1000
TIME_STEP = 0.1
PARTICLE_RADIUS = 0.05
CIRCLE_CENTER = np.array([0, 0])
CIRCLE_RADIUS = 5
FRAME_COUNT = 2000

# Initialize particles
particles = {
    'position': np.random.rand(NUM_PARTICLES, 2) * 10 - 5,
    'velocity': np.random.randn(NUM_PARTICLES, 2)
}

def update_particles(particles):
    # Update particle positions
    particles['position'] += particles['velocity'] * TIME_STEP

    # Check for collision with circle
    for i in range(NUM_PARTICLES):
        dist_to_center = np.linalg.norm(particles['position'][i] - CIRCLE_CENTER)
        if dist_to_center < CIRCLE_RADIUS + PARTICLE_RADIUS:
            # Reflect velocity
            direction = (particles['position'][i] - CIRCLE_CENTER) / dist_to_center
            particles['velocity'][i] -= 2 * np.dot(particles['velocity'][i], direction) * direction

def animate(frame):
    update_particles(particles)
    plt.cla()
    circle = plt.Circle(CIRCLE_CENTER, CIRCLE_RADIUS, color='r', fill=False)
    plt.gca().add_patch(circle)
    plt.scatter(particles['position'][:, 0], particles['position'][:, 1])
    plt.xlim(-10, 10)
    plt.ylim(-10, 10)
    plt.gca().set_aspect('equal', adjustable='box')

# Create animation
fig = plt.figure()
ani = FuncAnimation(fig, animate, frames=FRAME_COUNT, interval=100)

# Save the animation as a GIF
ani.save('particle_animation.gif', writer='pillow')
